# Initialization

Test notebook for the C-MAPPS benchmark. Test different MLP architectures. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers
from keras.layers import LSTM

import tensorflow as tf

/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
print(tf.__version__)
# print(tf.__version__sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)))

1.10.1


# Define architectures

Define each one of the different architectures to be tested.

In [3]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y


def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y

In [4]:
def tf_compiled_model(input_shape, output_shape):
    
    tf.reset_default_graph()
    X, y = create_placeholders(input_shape, output_shape)
    
    y_pred = tf_model(X)
    cost = tf.losses.mean_squared_error(y, y_pred)
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(total_cost)
    
    return {'X_placeholder':X, 'y_placeholder':y, 'y_pred':y_pred, 'cost':cost, 'total_cost':total_cost, 'optimizer':optimizer}

In [5]:
#Define the usable models for this notebook

#models = {'shallow-20-20':RULmodel_SN_4}

#models = {'shallow-250-100':RULmodel_SN_4, 'shallow-100-50':RULmodel_SN_1, 'shallow-50-20':RULmodel_SN_2,
#          'shallow-20-20':RULmodel_SN_3, 'shallow-20':RULmodel_SN_5, 'shallow-10':RULmodel_SN_6}
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Process Data

In [6]:
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 128

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, 
                                  window_size, window_stride)

# Build the model

In [7]:
tModel = SequenceTunableModelRegression('ann-20-20', None, lib_type='keras', data_handler=dHandler_cmaps)

# Load Data

In [8]:
#For LSTM
# tModel.data_handler.data_scaler = min_max_scaler
# tModel.data_scaler = None

#For ANN
tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 128

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[[128.]
 [128.]
 [128.]
 [128.]
 [128.]]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...  0.09090909  0.24409449
 

# Test on dataset 1

In [9]:
iterations = 10
tModel.epochs = 200
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
num_features = len(selected_features)

windowSize = 24
windowStride = 1
constRul = 128

file = open("results/MLP/ResultsDataset_1_tf.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')

models = {'ann-20-20':tf_compiled_model}

for key, model_def in models.items():
  
    print("For model "+str(key))
    #file.write("For model "+str(key)+'\n\n')
  
    for i in range(1,2):

        dataset = i
        print("Computing for dataset "+str(i))
        #file.write("Computing for dataset "+str(i)+'\n\n')
      
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
      
        input_shape = windowSize*num_features #For simple ANN
      
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = windowSize
        tModel.data_handler.sequence_stride = windowStride
        tModel.data_handler.max_rul = constRul
        tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
        #tModel.print_data()

        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            model = model_def(input_shape, 1)
            tModel.change_model('ModelRUL_SN_1', model, 'tensorflow')
            #model = get_compiled_model(model_def, input_shape, model_type='ann')

            sess = tf.Session()
            tModel.train_model(verbose=0, tf_session=sess)
            #tModel.train_model(learningRate_scheduler=lrate, verbose=0)
            tModel.evaluate_model(['rhs', 'rmse'], round=2, tf_session=sess)
            sess.close()
            #print("scores")
          
            #print(j)

            cScores = tModel.scores
            #rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
          
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time

        print("Results for model " + key)
  
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
          
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
        
file.close()

For model ann-20-20
Computing for dataset 1
Epoch:Final cost_reg= 211.684462653 cost= 182.337556627
tensorflow test
Epoch:Final cost_reg= 203.018555959 cost= 170.390945435
tensorflow test
Epoch:Final cost_reg= 208.612341563 cost= 176.371859656
tensorflow test
Epoch:Final cost_reg= 208.403184679 cost= 175.431611379
tensorflow test
Epoch:Final cost_reg= 204.973952823 cost= 172.253301409
tensorflow test
Epoch:Final cost_reg= 213.966761695 cost= 183.557486640
tensorflow test
Epoch:Final cost_reg= 210.278204176 cost= 181.552694109
tensorflow test
Epoch:Final cost_reg= 207.504407247 cost= 175.817009396
tensorflow test
Epoch:Final cost_reg= 207.287563324 cost= 177.869756911
tensorflow test
Epoch:Final cost_reg= 212.729979197 cost= 179.608186510
tensorflow test
Results for model ann-20-20
DescribeResult(nobs=10, minmax=(array([14.59897257]), array([15.39285549])), mean=array([14.98109115]), variance=array([0.05767564]), skewness=array([0.16166676]), kurtosis=array([-0.65879159]))
DescribeResul

# Test on all Datasets

In [11]:
datasets = [1,2,3,4]
iterations = 10
tModel.epochs = 150
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
window_sizes = {1:17,2:17,3:17,4:17}
strides = {1:1,2:1,3:1,4:1}
max_ruls = {1:129, 2:129, 3:129, 4:129}
num_features = len(selected_features)

input_shape = None
models = {'shallow-20-20':RULmodel_SN_4}

#For each model
for key, model_def in models.items():
    file = open("results/MLP/ResultsDatasets_singleSet"+key+".csv", "w")
    csvfile = csv.writer(file, lineterminator='\n')
    
    print("Generating statistics for model " + key)

    #For each dataset
    for i in range(1,5):
        
        print("Working on dataset " + str(i))
        
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
        
        input_shape = window_sizes[i]*num_features #For simple ANN
        #input_shape = (window_sizes[i],num_features) #For RNN
        
        print(input_shape)
        
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = window_sizes[i]
        tModel.data_handler.sequence_stride = strides[i]
        tModel.data_handler.max_rul = max_ruls[i]
        tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
        #tModel.print_data()
        
        #tModel.print_data()
        
        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            model = get_compiled_model(model_def, input_shape, model_type='ann')

            tModel.change_model(key, model, 'keras')
            tModel.train_model(learningRate_scheduler=lrate, verbose=0)
            tModel.evaluate_model(['rhs', 'rmse'], round=2)
            #print("scores")
            
            #print(j)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
            
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time
            
        print("Results for model " + key)
    
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
            
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
    
    file.close()

Generating statistics for model shallow-20-20
Working on dataset 1
238
100/100 [==============================] - 1s 10ms/step
Results for model shallow-20-20
DescribeResult(nobs=10, minmax=(array([16.74604431]), array([17.23861943])), mean=array([17.06463444]), variance=array([0.0325017]), skewness=array([-0.88353254]), kurtosis=array([-0.61935145]))
DescribeResult(nobs=10, minmax=(array([8.08271055]), array([9.26545247])), mean=array([8.64814575]), variance=array([0.20635946]), skewness=array([-0.04830206]), kurtosis=array([-1.50147762]))
DescribeResult(nobs=10, minmax=(array([17.28190351]), array([20.75944268])), mean=array([18.59064881]), variance=array([1.26079365]), skewness=array([0.53965178]), kurtosis=array([-0.70071753]))
Working on dataset 2
238
259/259 [==============================] - 1s 4ms/step
Results for model shallow-20-20
DescribeResult(nobs=10, minmax=(array([29.77055112]), array([30.12278605])), mean=array([29.9410427]), variance=array([0.01371166]), skewness=arra